In [2]:
import os
import subprocess
import time
import sys
import pandas as pd
module_path = "/data/luojaa/eukgen/"
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from core_functions.helper_functions import swarm_submit_with_lock, fasta_to_dict, dict_to_fastas, dict_to_fasta_len
from paths_and_parameters import clean_swarm_opts, proteinfer_swarm_opts, exe_reannotation

## uniprot set

### CLEAN

In [3]:
uniprot_fasta = "/data/tobiassonva/data/databases/reviewed_EC_numbers/uniprotkb_reviewed_and_published_with_EC.fasta"
# moved and replaced , with ;
        

In [8]:
in_fasta = "/data/luojaa/CLEAN_processing/uniprot/uniprotkb_quality_ec.fasta"
# result = subprocess.run(f"wc -l {in_fasta}".split(" "), capture_output = True)
# numlines = int(result.stdout.decode('ascii').split(" ")[0])

fasta_chunked_path = "/data/luojaa/CLEAN_processing/uniprot/INPUT/uniprotkb_quality_ec.fasta"
uniprot_fasta_dict = fasta_to_dict(file = in_fasta)
dict_to_fastas(uniprot_fasta_dict, fasta_chunked_path, 80)

60600

In [9]:
uniprot_df = pd.read_csv("/data/tobiassonva/data/databases/reviewed_EC_numbers/uniprotkb_reviewed_and_published_with_EC.tsv", sep = "\t")

In [17]:
uniprot_df.fillna("None", inplace = True)

In [36]:
len(uniprot_df["Entry Name"].dropna())

4722

### CLEAN lscratch

In [9]:
# root_fastas = "/data/luojaa/CLEAN_processing/uniprot/INPUT/"
# root_out = "/data/luojaa/CLEAN_processing/uniprot/OUTPUT/"
root_fastas = "/data/luojaa/CLEAN_processing/repseq/INPUT/"
root_out = "/data/luojaa/CLEAN_processing/repseq/OUTPUT/"
fastas = os.listdir(root_fastas)

clean_exe = "/data/luojaa/CLEAN_processing/CLEAN_infer_fasta.lscratch.py"

In [10]:
# # processing the fastas
# for fasta in fastas:
#     fasta = f"{root_fastas}{fasta}"
#     dict_fasta = fasta_to_dict(file=fasta)
#     dict_to_fasta_len(dict_fasta, write_file = fasta, verbose=False, maxlen=1021)

In [11]:
# format swarm file
print("formatting swarm file")
os.system(f"mkdir {root_out}tmp/")
swarmfile = root_out + 'tmp/clean_lscratch.swarm'
swarm = open(swarmfile, 'w')

# format swarm header for submission
swarm.write(''.join([f'#SWARM --{key} {value}\n' for key, value in clean_swarm_opts.items()]))

for fasta in fastas:
    fasta_path = f"{root_fastas}{fasta}"
    CLEAN_cmd = f"python3 -u {clean_exe} --fasta_data {fasta_path} \n"
    swarm.write(CLEAN_cmd)
    
swarm.close()
#os.system(f"rm {root_fastas}*tmp*")

formatting swarm file


mkdir: cannot create directory ‘/data/luojaa/CLEAN_processing/repseq/OUTPUT/tmp/’: File exists


### proteinfer

In [4]:
root_fastas = "/data/luojaa/proteinfer/annot_microcosms/repseq_fastas/"
root_out = "/data/luojaa/proteinfer/annot_microcosms/repseq_annots/"
fastas = os.listdir(root_fastas)

# format swarm file
print("formatting swarm file")
os.system(f"mkdir {root_out}tmp/")
swarmfile = root_out + 'tmp/reannotate_proteinfer.swarm'
swarm = open(swarmfile, 'w')

# format swarm header for submission
swarm.write(''.join([f'#SWARM --{key} {value}\n' for key, value in proteinfer_swarm_opts.items()]))

for fasta in fastas:
    fasta_path = f"{root_fastas}{fasta}"
    outpath = f"{root_out}{fasta}".replace(".fasta",".tsv")
    proteinfer_cmd = f"python3 -u {exe_reannotation} -i {fasta_path} -o {outpath} -reporting_threshold .01 \n"
    swarm.write(proteinfer_cmd)
    
swarm.close()
#os.system(f"rm {root_fastas}*tmp*")

formatting swarm file


### CLEAN_old

In [22]:
root_fastas = "/data/luojaa/proteinfer/annot_microcosms/repseq_fastas/"
clean_fastas = "/data/luojaa/CLEAN/app/data/inputs/"
clean_exe = "/data/luojaa/CLEAN/app/CLEAN_infer_fasta.py"
root_out = "/data/luojaa/CLEAN/app/results/"
fastas = os.listdir(root_fastas)
proteinfer_swarm_opts["job-name"] = "CLEAN"
proteinfer_swarm_opts["gb-per-process"] = "40"

In [23]:
# format swarm file
print("formatting swarm file")
os.system(f"mkdir {root_out}tmp/")
swarmfile = root_out + 'tmp/reannotate_clean.swarm'
swarm = open(swarmfile, 'w')

# format swarm header for submission
swarm.write(''.join([f'#SWARM --{key} {value}\n' for key, value in proteinfer_swarm_opts.items()]))

for fasta in fastas:
#    fasta_chunk = f"{root_fastas}{fasta}"
#     tmppath = f"{root_fastas}{fasta}.tmp"
#     with open(tmppath, "w") as tmp:
#         process = subprocess.Popen(f"sed -e s/\*//g {fasta_chunk}".split(" "), stdout = tmp)
#         process.wait()
#     process = subprocess.Popen(f"sed -i -e s/>/\\n>/g {tmppath}".split(" "))
#     process.wait()
#     with open(fasta_chunk, "w") as inp:
#         process = subprocess.Popen(f"sed -e /^[[:space:]]*$/d {tmppath}".split(" "), stdout = inp)
#         process.wait()
#    outpath = f"{root_out}{fasta}".replace(".fasta",".tsv")
    filename = fasta.strip(".fasta")
    CLEAN_cmd = f"python3 -u {clean_exe} --fasta_data {filename} \n"
    swarm.write(CLEAN_cmd)
    
swarm.close()
#os.system(f"rm {root_fastas}*tmp*")

formatting swarm file


mkdir: cannot create directory ‘/data/luojaa/CLEAN/app/results/tmp/’: File exists


In [5]:
microcosms_200enzymes = ['EP00956P017406',
 'EP00416P004364',
 'EP00828P047151',
 'EP00168P017972',
 'EP00472P014313',
 'EP01068P004615',
 'EP00048P006311',
 'EP00386P000866',
 'EP00429P112076',
 'EP00128P014172',
 'EP00451P049272',
 'EP01084P066589',
 'EP00152P004077',
 'XP_009048761.1',
 'EP00321P015807',
 'EP00898P000424',
 'EP00473P003522',
 'EP00931P120895',
 'EP00078P024396',
 'EP00924P004554',
 'EP01033P007035',
 'EP00741P022646',
 'EP01132P002973',
 'EP00750P001483',
 'EP00901P003718',
 'EP00085P025172',
 'XP_024309530.1',
 'EP00242P009817',
 'EP01043P065139',
 'EP00242P012213',
 'EP00413P011630',
 'EP00449P064556',
 'EP00041P033630',
 'EP00161P006019',
 'EP00327P050744',
 'EP01028P012630',
 'EP00247P004037',
 'EP00747P021543',
 'EP00013P004924',
 'EP00277P016487',
 'EP00932P000555',
 'EP00052P003084',
 'EP00474P001662',
 'EP01037P013534',
 'EP00292P009332',
 'EP00930P003442',
 'EP00747P192015',
 'EP00083P000008',
 'EP00008P011598',
 'EP00119P003428',
 'EP01038P006925',
 'EP00440P044931',
 'EP00515P008351',
 'EP00694P008262',
 'EP00440P036185',
 'EP00756P006435',
 'EP00264P084766',
 'EP00831P061494',
 'EP01035P040547',
 'EP00705P035375',
 'EP00416P023947',
 'EP00137P004576',
 'EP00304P007931',
 'EP00852P003688',
 'EP00100P001899',
 'EP00540P001622',
 'EP00049P023263',
 'EP00327P033321',
 'EP00881P000103',
 'EP00899P002709',
 'NP_013042.1',
 'EP00411P117219',
 'EP00985P003143',
 'EP00930P010586',
 'EP00048P018733',
 'EP00184P005355',
 'EP00047P021798',
 'EP01133P021481',
 'EP01094P020250',
 'EP00448P038236',
 'EP00119P016044',
 'EP01034P028265',
 'EP01052P014632',
 'EP01116P008898',
 'EP01066P011725',
 'EP00448P038131',
 'EP00702P001599',
 'EP01065P020638',
 'EP00584P004970',
 'EP01052P020941',
 'EP00264P042915',
 'EP00653P000934',
 'EP00902P019547',
 'EP00874P018117',
 'EP00091P005465',
 'EP00906P035832',
 'EP00392P015738',
 'EP01133P025146',
 'EP00658P054115',
 'EP00760P028411',
 'EP00237P015545',
 'EP01040P025643',
 'EP00593P008204',
 'EP00435P077613',
 'EP00926P002667',
 'EP01083P083081',
 'EP00151P025359',
 'EP00104P041324',
 'EP00168P000129',
 'EP00320P003358',
 'EP00747P066194',
 'EP01116P006431',
 'EP00906P015660',
 'EP00456P001876',
 'EP00729P004296',
 'EP00379P001824',
 'EP00942P003299',
 'EP00650P000063',
 'EP00429P139209',
 'EP00650P010563',
 'EP00750P011164',
 'EP00532P012511',
 'EP00294P001629',
 'EP00755P013348',
 'EP00427P035385',
 'EP00996P003673',
 'EP00815P011902',
 'EP00630P009299',
 'EP00638P008454',
 'XP_001311145.1',
 'EP00028P005065',
 'EP00093P008706',
 'EP00702P003265',
 'EP00929P015989',
 'EP00615P000761',
 'EP01137P019006',
 'EP01025P062051',
 'EP00592P023219',
 'EP00727P001599',
 'EP00092P026864',
 'EP01089P009709',
 'EP00609P011088',
 'EP00562P033516',
 'EP00879P027405',
 'EP01046P065782',
 'EP00642P002912',
 'EP00279P013884',
 'EP00250P008815',
 'EP00903P020570',
 'EP00586P006560',
 'EP00001P162079',
 'EP00462P024800',
 'EP00953P010372',
 'EP01027P008826',
 'EP00406P005694',
 'EP00752P015698',
 'EP01053P009017',
 'EP01083P075870',
 'EP00566P017299',
 'EP00092P079308',
 'NP_586244.1',
 'EP00057P008717',
 'EP00468P006976',
 'EP00751P012726',
 'EP00961P018891',
 'XP_005761688.1',
 'EP01102P002161',
 'EP00623P002564',
 'EP01042P013226',
 'EP00630P011952',
 'EP00286P025649',
 'EP00327P018531',
 'EP00697P003493',
 'EP00042P031698',
 'EP00112P006728',
 'EP00441P007813',
 'EP00421P072821',
 'EP00736P013596',
 'EP00057P010553',
 'EP01138P009754',
 'EP01085P001863',
 'EP00608P011520',
 'EP00913P001883',
 'EP00667P018896',
 'EP01030P016725',
 'EP00600P003182',
 'EP00297P003795',
 'XP_004338622.1',
 'EP01031P051734',
 'EP01034P045733',
 'EP01037P018018',
 'EP01136P007224',
 'EP00548P013232',
 'EP01041P024678',
 'EP00276P003263',
 'EP01041P028335',
 'EP00228P000922',
 'EP00820P012879',
 'EP00425P001732',
 'EP01053P003446']

In [13]:
glycolysis_m = ['EP00527P005307',
 'EP00449P046851',
 'EP00137P000668',
 'EP00297P006020',
 'EP01036P000861',
 'EP00928P001802',
 'EP00401P052901',
 'EP00978P019064',
 'EP01034P032049',
 'EP00703P005080',
 'EP01051P022176',
 'EP00796P010538',
 'EP00929P071679',
 'EP00404P045045',
 'EP00149P002640',
 'EP00899P015215',
 'EP00650P010264',
 'EP00036P009887',
 'EP00249P018185',
 'EP00104P036677',
 'EP00828P033823',
 'EP00299P017403',
 'EP00429P064283',
 'EP01065P042669',
 'EP00956P034078',
 'EP00411P047591',
 'EP01048P032169',
 'EP00750P018662',
 'EP01037P042805',
 'EP00421P001545',
 'EP00271P015981',
 'EP00537P018534',
 'EP00514P007840',
 'EP00450P028435',
 'EP01071P010579',
 'EP00419P039549',
 'EP01032P005185',
 'EP00242P012213',
 'EP00896P018378',
 'EP00879P020714',
 'EP00041P033630',
 'EP01037P026313',
 'EP00592P027762',
 'EP01137P008667',
 'EP00465P004832',
 'EP00811P036611',
 'EP00303P000755',
 'EP00277P016487',
 'EP00619P008385',
 'EP00898P000779',
 'EP01032P038900',
 'EP00035P031713',
 'EP00961P272016',
 'EP01016P056673',
 'EP00063P066385',
 'EP00884P021724',
 'EP00442P020291',
 'EP00925P014598',
 'EP00074P078910',
 'EP01026P048277',
 'EP01090P003192',
 'EP00136P001771',
 'EP00053P019257',
 'EP01047P013849',
 'EP00825P021096',
 'EP00852P001018',
 'EP00515P012205',
 'EP00820P010734',
 'EP00276P007785',
 'EP00189P008809',
 'EP01010P003062',
 'EP01047P076025',
 'EP00439P014581',
 'EP00510P000546',
 'EP00397P050253',
 'XP_020044643.1',
 'EP01084P229107',
 'EP00439P088743',
 'EP00304P005735',
 'EP00435P028350',
 'EP00980P018513',
 'EP00074P075135',
 'EP00901P004205',
 'EP00878P014314',
 'EP00104P028049',
 'EP00264P057993',
 'EP00747P171382',
 'EP00967P054697',
 'EP00264P141125',
 'EP01114P009938',
 'EP00084P001529',
 'EP00035P002515',
 'EP00163P012543',
 'EP00511P003537',
 'EP00756P026020',
 'EP01041P018052',
 'EP00563P015401',
 'EP00054P004199',
 'EP00634P013541',
 'EP00871P004814',
 'EP01133P021481',
 'EP00232P011923',
 'EP00356P013706',
 'EP00315P016170',
 'EP01037P003939',
 'EP00123P004527',
 'EP01105P028472',
 'EP00557P003143',
 'EP00662P016361',
 'EP00003P022107',
 'EP00897P008568',
 'EP01132P019536',
 'EP00406P038559',
 'EP01052P020941',
 'EP00168P018762',
 'EP00232P028993',
 'EP00303P007657',
 'EP00271P012915',
 'EP00877P007182',
 'EP00932P018031',
 'EP00263P001882',
 'EP01017P007234',
 'XP_657874.1',
 'EP01083P143712',
 'EP01134P008213',
 'EP00195P014777',
 'EP00494P002295',
 'EP00621P003234',
 'EP00456P034547',
 'EP01108P007086',
 'NP_788764.1',
 'EP00222P003460',
 'EP00406P041464',
 'EP00672P028238',
 'EP01092P015481',
 'EP00532P009663',
 'EP00884P000292',
 'EP00908P016189',
 'EP00681P006252',
 'EP00320P003358',
 'EP00829P015469',
 'EP00661P000700',
 'EP01037P024253',
 'XP_024533280.1',
 'XP_031498832.1',
 'EP01117P007056',
 'EP01041P028216',
 'EP00232P030721',
 'EP00295P020620',
 'EP00429P139209',
 'EP00741P008320',
 'EP00703P002120',
 'EP01036P016030',
 'EP00659P020590',
 'EP00191P007747',
 'EP00438P026313',
 'EP00914P004235',
 'EP00419P014545',
 'EP00404P026775',
 'EP00832P007568',
 'XP_016859929.1',
 'EP00624P002364',
 'EP00759P050058',
 'EP00420P018590',
 'EP00882P002512',
 'EP00158P010171',
 'EP01114P017101',
 'EP01036P052480',
 'EP01029P023018',
 'EP00864P003644',
 'EP00929P015989',
 'EP00295P025046',
 'EP00231P007254',
 'EP00656P018215',
 'EP01035P008247',
 'EP01090P000336',
 'EP00870P002221',
 'EP01051P017204',
 'EP00592P023219',
 'EP00411P025937',
 'EP00760P034085',
 'EP00136P003596',
 'EP00092P026864',
 'EP00800P006030',
 'EP01035P029197',
 'EP00250P008815',
 'EP00427P002134',
 'EP00860P001013',
 'EP00747P193451',
 'EP00287P021250',
 'EP00893P001025',
 'EP00913P010278',
 'XP_007708955.1',
 'EP00303P002862',
 'EP00828P030925',
 'EP00638P003560',
 'EP00406P005694',
 'EP00388P038175',
 'XP_005794406.1',
 'EP00449P006696',
 'EP00764P003383',
 'EP00906P009477',
 'EP01126P056738',
 'EP00450P038948',
 'EP00980P004243',
 'EP00825P030661',
 'EP01071P005187',
 'EP00904P013850',
 'EP00376P007078',
 'EP00414P003148',
 'EP00850P014668',
 'EP00266P021809',
 'EP01031P000897',
 'EP00884P022630',
 'EP00490P009897',
 'EP00925P012180',
 'EP00286P025649',
 'EP00211P004062',
 'EP00414P000575',
 'EP00301P019341',
 'EP01133P021881',
 'EP00667P018842',
 'EP00264P219639',
 'EP00956P019851',
 'EP00008P015655',
 'EP00033P007857',
 'EP00036P013193',
 'EP00137P005934',
 'EP00139P004449',
 'EP00440P033281',
 'EP01132P021787',
 'EP00747P184313',
 'EP00658P038559',
 'EP00310P020786',
 'EP00543P008555',
 'EP00658P070392',
 'EP00742P011898',
 'EP00638P006667',
 'EP00166P002679',
 'EP01130P014384',
 'EP00293P017034',
 'EP00553P002328',
 'EP00656P027363',
 'EP00928P057620',
 'EP00753P018640',
 'EP01032P017040',
 'EP00548P013232',
 'EP01086P016048',
 'EP00120P007748',
 'EP00287P003944',
 'EP00435P000097',
 'EP00927P019965',
 'EP01062P020953',
 'EP00613P033126',
 'EP00802P034667',
 'EP01001P013054',
 'EP00051P025233',
 'EP00304P004700',
 'EP00421P010910',
 'EP00440P028648',
 'EP00151P012936',
 'EP00611P014087',
 'EP00455P046230',
 'NP_077306.3',
 'EP00638P021733',
 'EP00429P080798',
 'EP00747P076513',
 'EP01063P002746',
 'EP00404P001377',
 'EP00163P026298',
 'EP00882P025785',
 'EP00903P002099',
 'EP00618P036919',
 'EP01041P015347',
 'EP00028P010433',
 'EP00859P003157',
 'EP00851P008456',
 'EP00147P010094',
 'EP00480P004361',
 'EP00833P009205',
 'EP00271P016318',
 'EP00264P193567',
 'XP_012491767.1',
 'EP00027P009810',
 'EP00429P129283',
 'EP00747P212613',
 'EP00223P002426',
 'EP00264P072572',
 'EP00619P015604',
 'EP00260P008961',
 'EP00919P047259',
 'EP01083P137702',
 'EP01025P065204',
 'EP00406P080305',
 'EP01006P048098',
 'EP01095P010727',
 'EP00026P010229',
 'EP00392P007764',
 'EP00939P005184',
 'EP00161P022828',
 'EP00863P015264',
 'EP00611P018014',
 'EP00443P057666',
 'EP00002P015014',
 'EP00665P008984',
 'EP00913P005868',
 'EP00887P002764',
 'EP00933P023908',
 'EP00703P003497',
 'EP00667P010370',
 'EP00922P048472',
 'EP00579P010746',
 'EP00321P012419',
 'EP00232P014959',
 'EP00672P020331',
 'EP00644P019765',
 'EP00029P007967',
 'EP00166P000315',
 'EP01092P002812',
 'EP00264P218674',
 'EP00658P014011',
 'EP00669P014570',
 'EP00162P018823',
 'EP00964P080958',
 'EP00224P000921',
 'EP00443P072525',
 'EP00512P008892',
 'EP00569P010692',
 'EP00930P018128',
 'EP00630P005454',
 'EP00439P087710',
 'EP01037P016020',
 'EP00998P005448',
 'EP00264P003798',
 'EP00490P001011',
 'EP00128P017112',
 'EP00339P018677',
 'EP01031P047366',
 'EP00659P005274',
 'EP00500P013839',
 'EP00759P017600',
 'XP_001744962.1',
 'EP00516P009963',
 'EP01028P003443',
 'EP00211P014996',
 'EP00307P007302',
 'EP00659P012242',
 'EP00521P012403',
 'EP00268P052389',
 'EP00850P015694',
 'EP01036P040203',
 'EP00051P030027',
 'EP00802P016671',
 'EP00449P023489',
 'EP00302P039507',
 'EP00820P022009',
 'EP00759P010601',
 'EP00344P001763',
 'EP00295P038465',
 'EP00055P018435',
 'PTQ39957.1',
 'EP00078P026822',
 'EP00302P005383',
 'EP01028P009307',
 'EP01068P002234',
 'EP01037P032302',
 'EP01114P017586',
 'EP01114P018949',
 'EP01083P261720',
 'EP00100P000576',
 'EP00929P117855',
 'EP01118P003532',
 'EP00831P003848',
 'EP00401P041588',
 'EP01122P014414',
 'EP00009P000201',
 'EP00449P011557',
 'EP00981P002339',
 'EP00434P039200',
 'EP01041P018193',
 'EP00668P033549',
 'EP00831P056873',
 'EP00004P001574',
 'EP01083P209222',
 'EP00401P063515',
 'EP00828P015209',
 'EP00750P006362',
 'EP01084P175052',
 'EP00746P039705',
 'EP00703P000087',
 'EP00443P077850',
 'EP00826P035541',
 'EP00421P008001',
 'EP00187P013340',
 'EP01033P007442',
 'EP00408P037099',
 'EP00533P008502',
 'EP00411P060113',
 'EP00745P001199',
 'EP00858P001650',
 'EP00427P044245',
 'XP_031505906.1',
 'EP01128P009212',
 'EP00930P001603',
 'EP01035P014622',
 'EP01155P019627',
 'EP00640P002503',
 'EP01029P024147',
 'XP_026455769.1',
 'EP00040P002640',
 'EP00211P003360',
 'EP00428P067699',
 'EP00456P030495',
 'EP00462P007365',
 'EP00566P013665',
 'EP00302P044725',
 'EP00747P183908',
 'EP00740P022843',
 'EP00551P019332',
 'EP00670P013026',
 'EP01034P027643',
 'EP00897P008338',
 'EP00908P022752',
 'EP00120P001434',
 'EP00931P075880',
 'EP00188P004519',
 'EP01155P024159',
 'EP00820P017522',
 'EP00354P001143',
 'EP00414P001044',
 'EP00019P009225',
 'EP01059P026247',
 'EP00656P026669',
 'EP00324P005019',
 'EP01052P003903',
 'EP00971P266131',
 'EP01006P015924',
 'EP01156P011214',
 'EP00382P002879',
 'EP00731P032209',
 'EP00060P027949',
 'EP00202P003875',
 'EP00434P039519',
 'EP00932P014955',
 'EP01036P044298',
 'EP00960P069008',
 'EP00315P002915',
 'EP00264P144116',
 'EP01105P028809',
 'EP00762P002282',
 'EP00285P001632',
 'EP00831P033204',
 'EP00049P002247',
 'EP00824P007216',
 'EP00042P054044',
 'EP00466P013778',
 'EP00193P010652',
 'NP_179320.1',
 'EP00659P015729',
 'EP00429P052369',
 'EP01155P019067',
 'XP_012896513.1',
 'EP00850P024670',
 'EP00527P007603',
 'EP01028P011405',
 'RWR77818.1',
 'EP01084P222197',
 'EP00829P018131',
 'EP00825P024626',
 'EP00504P004023',
 'EP01130P003384',
 'EP00439P025027',
 'EP01132P017029',
 'EP00411P065933',
 'EP00095P010123',
 'EP01037P007210',
 'EP01013P008862',
 'EP00264P172149',
 'EP01088P008753',
 'XP_008667287.1',
 'EP01002P000474',
 'EP00344P000497',
 'EP01035P015002',
 'EP00038P002856',
 'EP00193P021999',
 'EP00212P005619',
 'EP00800P010914',
 'RWR78054.1',
 'EP01061P008440',
 'EP00759P042646',
 'EP00535P006867',
 'EP01065P028877',
 'EP00792P004555',
 'EP00170P005713',
 'EP00746P059394',
 'EP01043P009313',
 'EP01065P027190',
 'EP00326P002140',
 'EP00928P037542',
 'EP01084P198410',
 'EP00358P005532',
 'EP00443P058305',
 'EP00408P057050',
 'EP01115P004456',
 'EP01091P011926',
 'EP00283P008560',
 'EP00405P034448',
 'EP00480P028604',
 'EP01133P002300',
 'EP00448P021259',
 'EP00033P000756',
 'EP01036P054615',
 'EP00729P013688',
 'EP00902P017976',
 'EP00429P040517',
 'EP01030P020506',
 'EP00996P003147',
 'EP01065P041480',
 'EP01032P018035',
 'EP00680P011213',
 'EP01110P011116',
 'EP00161P019290',
 'EP00191P018166',
 'EP00416P011338',
 'EP00408P024521',
 'EP00203P006222',
 'EP00264P074206',
 'EP01041P014385',
 'EP00126P001918',
 'EP00730P014497',
 'EP00344P011901',
 'EP00919P027414',
 'EP00931P041735',
 'EP00769P001727',
 'EP00040P039143',
 'EP00879P008936',
 'EP00599P010883',
 'EP00594P014576',
 'EP01032P013777',
 'EP00746P127082',
 'EP00229P006480',
 'EP00392P008945',
 'EP00875P010812',
 'EP00511P006289',
 'EP00163P023491',
 'EP00242P006509',
 'EP00587P004147',
 'EP00129P012304',
 'EP00241P015007',
 'EP00816P013180',
 'EP00843P019361',
 'EP00832P000410',
 'EP00882P003600',
 'EP00169P000872',
 'EP01084P195551',
 'EP00746P160991',
 'EP00084P008552',
 'EP00562P002573',
 'EP01006P003508',
 'EP01122P002068',
 'GBG74526.1',
 'EP01133P000805',
 'EP00316P013007',
 'EP00331P030427',
 'EP00264P017469',
 'EP00602P007659',
 'EP01037P014083',
 'EP01034P015384',
 'EP00957P116257',
 'EP00810P020272',
 'EP00030P003018',
 'EP00850P013345',
 'EP00310P016217',
 'EP00824P026435',
 'EP00255P005471',
 'EP00949P005119',
 'EP00630P001137',
 'EP00927P074161',
 'EP00443P102486',
 'EP00295P002061',
 'EP00445P051005',
 'EP00427P042318',
 'EP00804P015559',
 'EP01036P052305',
 'EP01083P165605',
 'EP01032P007950',
 'EP01065P036461',
 'EP00804P018175',
 'EP00750P003147',
 'EP01012P056422',
 'EP00036P018141',
 'EP00878P030040',
 'EP01090P014455',
 'EP00244P003123',
 'EP00620P014574',
 'EP00037P005347',
 'EP01126P042539',
 'EP00074P006000',
 'EP00475P037434',
 'EP00240P001384',
 'EP00931P102732',
 'EP00244P000692',
 'EP00633P006843',
 'EP00641P003629',
 'EP01118P013641',
 'EP00420P037647',
 'EP00633P001838',
 'EP00815P008578',
 'EP00878P025018',
 'EP00923P018850',
 'EP00429P131944',
 'EP00062P005793',
 'EP00927P001234',
 'EP00929P031801',
 'EP00456P019200',
 'EP01036P049167',
 'OAE29080.1',
 'EP00103P007631',
 'EP00164P000261',
 'EP00413P029388',
 'EP00599P016082',
 'EP00529P028538',
 'EP00350P004810',
 'EP00920P020406',
 'EP01032P022202',
 'EP00290P043511',
 'EP01022P023970',
 'EP00282P032807',
 'EP00312P028212',
 'EP00295P005085',
 'EP00698P019805',
 'EP01042P007405',
 'OAD06347.1',
 'EP00932P016170',
 'EP00549P020207',
 'EP00986P018311',
 'EP00893P003359',
 'EP01091P004912',
 'EP00132P000964',
 'EP00217P002972',
 'EP00808P023984',
 'EP01012P049984',
 'EP00429P080468',
 'EP01125P005289',
 'EP00435P035358',
 'EP00850P005911',
 'EP00167P004143',
 'EP00882P021872',
 'EP01116P021510',
 'EP00448P038349',
 'XP_001618265.2',
 'EP00264P085740',
 'EP00419P058975',
 'EP00639P023319',
 'EP00264P218672',
 'EP01066P014303',
 'EP00529P014261',
 'EP01117P001050',
 'EP00327P023131',
 'EP00802P005271',
 'EP00003P000417',
 'EP01032P024617',
 'EP00650P010945',
 'EP00277P032255',
 'EP00277P024332',
 'EP00237P004005',
 'EP01035P048732',
 'EP01037P027161',
 'EP00413P045347',
 'EP01133P024626',
 'XP_007400516.1',
 'EP00747P182349',
 'EP01093P018863',
 'EP01147P007311',
 'EP00444P042203',
 'EP00443P056711',
 'EP00831P046362',
 'NP_973729.1',
 'EP00274P004312',
 'EP00020P002543',
 'EP00639P019523',
 'EP01083P275383',
 'EP00945P001951',
 'EP00526P004326',
 'EP00435P077994',
 'EP00108P004022',
 'EP00747P003601',
 'EP00647P005306',
 'EP00230P010861',
 'EP00756P016554',
 'EP01121P019085',
 'EP01043P055072',
 'EP00276P025008',
 'EP00176P004363',
 'EP00358P000170',
 'EP00406P037316',
 'EP00326P042246',
 'EP00006P012143',
 'EP00279P019013',
 'EP00110P001649',
 'EP00923P018855',
 'EP00310P024816',
 'EP00671P006101',
 'EP00680P009116',
 'EP01048P022538',
 'EP00686P010703',
 'NP_001119199.1',
 'EP00295P006551',
 'EP00898P005659',
 'EP01123P001118',
 'EP00401P065722',
 'EP00448P037046',
 'EP00548P001613',
 'EP00708P002419',
 'EP00258P035526',
 'EP01034P039579',
 'EP01038P016768',
 'EP00429P074210',
 'EP00706P017065',
 'EP00429P018763',
 'EP00029P004101',
 'EP00971P138923',
 'EP00434P046359',
 'EP00143P002444',
 'EP00163P013896',
 'EP00052P014364',
 'EP00429P135605',
 'EP00302P043510',
 'EP00542P009942',
 'EP00882P011620',
 'EP00792P012912',
 'EP01022P001364',
 'EP00180P003741',
 'EP00445P067793',
 'EP00005P012197',
 'EP01024P045260',
 'EP00349P001474',
 'EP01053P005225',
 'EP01102P020506',
 'EP00232P025183',
 'EP00326P007847',
 'EP00982P013527',
 'EP00269P022277',
 'EP00842P004148',
 'EP00851P009374',
 'EP01036P003586',
 'EP00010P002398',
 'EP00232P003609',
 'EP00882P031567',
 'EP00747P201860',
 'EP00798P008027',
 'EP00857P005594',
 'EP00752P006639',
 'EP00386P003677',
 'EP01001P010098',
 'EP01035P006472',
 'EP01146P000906',
 'EP00849P002988',
 'EP00518P004241',
 'EP00408P019697',
 'EP00222P005193',
 'EP00877P004131',
 'EP01089P011886',
 'EP00413P004400',
 'EP00764P010655',
 'EP00920P040965',
 'EP00966P023101',
 'EP00486P013320',
 'EP00764P011271',
 'EP01099P005858',
 'EP00300P027865',
 'EP00835P005408',
 'EP01024P069440',
 'EP01066P001720',
 'EP00876P011863',
 'EP00187P004626',
 'EP01016P003446',
 'EP00380P025483',
 'EP00521P005001',
 'EP01124P003801',
 'EP01090P014180',
 'EP01119P011011',
 'EP00770P003181',
 'EP00031P001495',
 'EP00428P004541',
 'EP00755P032605',
 'EP00746P009149',
 'EP01034P013145',
 'EP00616P000758',
 'EP00672P017229',
 'EP01052P044182',
 'EP01060P005190',
 'EP01083P065872',
 'EP00029P011415',
 'EP00708P002176',
 'EP00945P001783',
 'EP00136P009082',
 'EP00326P039632']